In [ ]:
import pandas as pd
import re

language_corrections = {
    'Dari Persian': 'Persian',
    'Pashtu': 'Pashto',
    'Azerbaijani Turkic': 'Azerbaijani',
    'Dutch (Flemish)': 'Dutch',
    'Dzongkha': 'Dzongkha',
    'Bosnian': 'Serbo-Croatian',
    'Croatian': 'Serbo-Croatian',
    'Serbian': 'Serbo-Croatian',
    'Setswana': 'Tswana',
    'Bulgarian': 'Bulgarian',
    'French': 'French',
    'Standard Chinese (Mandarin/Putonghua)': 'Mandarin Chinese',
    'Yue (Cantonese)': 'Cantonese',
    'Arabic': 'Standard Arabic',
    'German': 'German',
    'Spanish': 'Spanish',
    'Italian': 'Italian',
    'Portuguese': 'Portuguese',
    'Russian': 'Russian',
    'Kazakh': 'Kazakh',
    'Latvian': 'Latvian',
    'Lithuanian': 'Lithuanian',
    'Macedonian': 'Macedonian',
    'Hungarian': 'Hungarian',
    'Nepali': 'Nepali',
    'Polish': 'Polish',
    'Urdu': 'Urdu',
    'Punjabi': 'Punjabi',
    'Norwegian': 'Norwegian',
    'Tigrinya': 'Tigrinya',
    'Finnish': 'Finnish',
    'Swedish': 'Swedish',
    'Tamil': 'Tamil',
    'Sinhala': 'Sinhala',
    'Maltese': 'Maltese',
    'Mandarin': 'Mandarin Chinese',
    'Hokkien': 'Min Nan Chinese',
    'Tswana': 'Tswana',
    'Xhosa': 'Xhosa',
    'Zulu': 'Zulu',
    'Sesotho': 'Sotho',
    'Shikomoro': 'Swahili',
    'Luxermbourgish': 'Luxembourgish',
    'Wu (Shanghaiese)': 'Wu Chinese',
    'Minbei (Fuzhou)': 'Min Bei Chinese',
    'Minnan (Hokkien-Taiwanese)': 'Min Nan Chinese',
    'Tibetan dialects': 'Tibetan',
    'Nepalese dialects': 'Nepali',
    'Persian and Persian dialects': 'Persian',
    'Turkic and Turkic dialects': 'Turkish',
    'Hassaniya Arabic': 'Standard Arabic',
    'Moroccan Arabic': 'Standard Arabic',
    'Luxermbourgish': 'Luxembourgish',
    'Romansch': 'Romansh',
    'Palauan': 'Palauan',
    'Tetum': 'Tetum',
    'Tok Pisin': 'Tok Pisin',
    'Bislama': 'Bislama',
    'Vietnamese': 'Vietnamese',
    'Tajik': 'Tajik',
    'Uzbek': 'Uzbek',
    'Hiri Motu': 'Hiri Motu',
    'English': 'English',
    'French': 'French',
    'German': 'German',
    'Russian': 'Russian',
    'Spanish': 'Spanish'
}

def clean_language(language_str):
    # Separar por comas y tomar el primer idioma
    primary_language = language_str.split(',')[0]
    # Eliminar texto con porcentaje o cualquier otro paréntesis
    primary_language = re.sub(r'\([^)]*\)|\d+%|[\d-]+\%|[\d-]+\%', '', primary_language).strip()
    # Ajuste adicional para manejar "Castilian Spanish" y "Spanish"
    if 'Castilian' in primary_language or 'Spanish' in primary_language:
        return 'Spanish'
    return primary_language

# Cargar los archivos CSV
top_languages_df = pd.read_csv('data/Top 100 Languages.csv')
countries_languages_df = pd.read_csv('data/countries-languages.csv')

# Normalización de los idiomas en la columna "Languages Spoken"
countries_languages_df['Cleaned Languages'] = countries_languages_df['Languages Spoken'].apply(lambda x: clean_language(x))

# Expansión de idiomas en filas individuales
expanded_df = countries_languages_df.assign(Language=countries_languages_df['Cleaned Languages'].str.split(', ')).explode('Language')

# Aplicar correcciones en la columna Language
expanded_df['Language'] = expanded_df['Language'].replace(language_corrections)

# Realizar la unión con el dataframe de los Top 100 Languages
merged_df_corrected = expanded_df.merge(top_languages_df, left_on='Language', right_on='Language', how='left')

# Filtrar el idioma principal por país basado en la coincidencia con los 100 idiomas más hablados
primary_language_df_corrected = merged_df_corrected.dropna(subset=['Total Speakers']).drop_duplicates(subset=['Country'], keep='first')

# Crear una columna para hablantes no nativos (second language speakers)
primary_language_df_corrected['Second Language Speakers'] = primary_language_df_corrected['Total Speakers'] - primary_language_df_corrected['Native Speakers']

# Agrupar los otros idiomas hablados por país
other_languages_df_corrected = expanded_df[~expanded_df['Language'].isin(top_languages_df['Language'])]
other_languages_df_corrected = other_languages_df_corrected.groupby('Country')['Language'].apply(lambda x: ', '.join(x)).reset_index()

# Hacer un merge para agregar los otros idiomas hablados al dataframe principal
final_df_corrected = primary_language_df_corrected[['Country', 'Language', 'Total Speakers', 'Native Speakers', 'Second Language Speakers', 'Origin']].merge(
    other_languages_df_corrected, on='Country', how='left')

# Renombrar las columnas de acuerdo a lo solicitado
final_df_corrected.rename(columns={
    'Country': 'Pais',
    'Language': 'Primary Language',
    'Language_y': 'Other Languages Spoken'
}, inplace=True)

# Guardar el resultado final
final_df_corrected.to_csv('data/Final_Languages_by_Country.csv', index=False)